In [ ]:
## Import all necessary packages

from urllib.request import urlopen
import json
import pandas as pd
import plotly.graph_objects as go


## Agenda
1. Define function to allow reading of Geojson data
2. Learn where to find spacial data and practice reading this into python
3. Create a simple map, plotting CCGs of your choice
4. Play around with editing the map colours and adding hospital locations on top
5. Create a heatmap for LSOAs within your CCG to show deprivation decile by LSOA

## 1. Define useful functions

In [ ]:
## This function allows you to read in the geojson file later on

def load_geojson(url, feature_name):
    with urlopen(url) as site:
        data = json.loads(site.read().decode())
        
    for k in range(len(data['features'])):
        data['features'][k]['id'] = data['features'][k]['properties'][feature_name]
        
    return data

## 2. Read in CCG level spacial data
- Geospacial data is found on geoportal, ran by the ONS, at the web address: https://geoportal.statistics.gov.uk/
- The CCG boundaries are found by navigating to Boundaries > Health boundaries > Clinical commissioning groups > 2021 boundaries
- The boundaries are given with different resolution qualities. Choosing ultra generalised will allow the code to run more efficiently

- Once you have clicked on the data set that you would like to use, an information box should appear on the left hand side of your page
- Click on 'I want to use this' in the blue bar at the bottom of the information box
- Select 'View API resources' and copy the link in the box called 'GeoJSON'
- Paste the link below, next to 'ccg_url ='
- Note: The links to the geojson data can change from time to time, so if this code does not work next time you run it, you may to need to check back on geoportal for a new link


In [ ]:
ccg_url = ""
ccg = load_geojson(ccg_url, 'CCG21NM')

ValueError: unknown url type: ''

Try running the ccg to have a look at the geojson data - you will notice that it is comprised of CCG names and their coordiantes

In [ ]:
ccg

## 3. Create a simple map of CCGs of your choice

In [ ]:
## Create a dummy data frame with the CCGs that you would like to map
## Note: the geojson data we downloaded is using the 2021 CCGs, therefore older ones 
    #(e.g. Redbridge/Havering will not work)
    
ccg_nm = pd.DataFrame(columns=['test'])
ccg_nm.loc['NHS Mid Essex CCG'] = 5
ccg_nm.loc['NHS West Essex CCG'] = 7
ccg_nm.loc['NHS North East Essex CCG'] = 10
#ccg_nm.loc['NHS North East London CCG'] = 10
ccg_nm


## Pass the data through the map making function
- Here we are using Choroplethmapbox to create a simple map
- If you have chosen non Essex CCGs, you will need to chage the mapbox centre to look to the right pace. This is done by adjusting the lat and long co-ordinates
- Un-comment out the North East London CCG (above), then re-run the map code below to see what happens

The documentation for Choroplethmapbox can be found here:
- https://plotly.com/python-api-reference/generated/plotly.graph_objects.Choroplethmapbox.html


In [ ]:
#This bit adds the CCG boundaries
# We are specifying locations in the locations parameter - only these are plotted
fig = go.Figure(go.Choroplethmapbox(geojson=ccg, locations=ccg_nm.index, z=ccg_nm.test,
                                    zmin=0, zmax=10,colorscale=['orange','white'],
                                    marker_opacity=1, marker_line_width=2))

# This is setting up the map - to change the area zoomed in on, change the lat and long
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7, mapbox_center = {"lat": 51.8717, "lon": 1.067965})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update(layout_coloraxis_showscale=False)

fig.show()

## Congratulations on making your first map!

## 4. Play around with editing the map visuals
- There are lots of things that you can adjust including:
    - Colour scale
    - Marker opacity and line width
    - The position of the map (using lat and lon) and how zoomed in it is
- Have a play around with changing some of the settings

#### Different mapbox styles can be found here:
- https://plotly.com/python/mapbox-layers/

#### The colorscale can use words, pre-set colour scales or RGB codes. For example, have a go at replacing colourscale with:
- colorscale = ['#27A5B8','white']
- colorscale = 'earth' 
- Or select one of the colours from the image below

![image.png](attachment:image.png)

In [ ]:
# Try changing the colour scale, line width or marker opacity
fig = go.Figure(go.Choroplethmapbox(geojson=ccg, locations=ccg_nm.index, z=ccg_nm.test,
                                    zmin=0, zmax=10,colorscale = ['orange','white'],
                                    marker_opacity=1, marker_line_width=2))

# Try changing the mapbox style or mapbox zoom 
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7, mapbox_center = {"lat": 51.8717, "lon": 1.067965})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update(layout_coloraxis_showscale=False)

fig.show()

## 5. Add hospital locations on top
- Save the hospital locations data file
- Read in the file
- Subset the data to include the hospitals that you are interested in plotting

In [ ]:
# CSV file with hospital lat longs
hospitals = pd.read_csv('all_hospital_locations.csv')

In [ ]:
# Subset the hospital location file to include just the hospitals that you want to include in your map
# You can add multiple hospitals by writing more in the final square bracket
Essex_hospitals = (hospitals.loc[hospitals['OrganisationName'].isin(["St Peter's Hospital"])])

In [ ]:
Essex_hospitals

In [ ]:
#This bit adds the CCG boundaries
fig = go.Figure(go.Choroplethmapbox(geojson=ccg, locations=ccg_nm.index, z=ccg_nm.test,
                                    zmin=0, zmax=10,colorscale=['orange','white'],
                                    marker_opacity=1, marker_line_width=2))


# This is adding the dots on the maps for your hospital locations
# This bit makes a large red dot
fig.add_trace(go.Scattermapbox(
        lat=Essex_hospitals['Latitude'],
        lon=Essex_hospitals['Longitude'],
        mode='markers',
        showlegend = False,
        marker=go.scattermapbox.Marker(
            size=11,
            color='red'
        )
    ))

# This bit puts a small white dot inside (just because I think it looks nice)
fig.add_trace(go.Scattermapbox(
         lat=Essex_hospitals['Latitude'],
         lon=Essex_hospitals['Longitude'],
         showlegend = False,
         marker=go.scattermapbox.Marker(
             size=7,
             color='white'
         )
    ))

# This is setting up the map - to change the area zoomed in on, change the lat and long
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=8, mapbox_center = {"lat": 51.8717, "lon": 1.067965})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update(layout_coloraxis_showscale=False)


fig.show()

## Excercise - have a go at adding another hospital location to your map
- Go back to 'Essex hospitals' where we subset the hospital locations file and add in another nearby hospital
- Re-run your map making code to see your new hospital plotted

### Read in CCG level data showing under 75 mortality for CVDs
Data can be found from the following source: 
https://digital.nhs.uk/data-and-information/publications/statistical/ccg-outcomes-indicator-set/march-2022/domain-1-preventing-people-from-dying-prematurely-ccg/1-2-under-75-mortality-from-cardiovascular-disease

## Exercise - Create a map showing CVD under 75 mortality for all CCGs in the country

In [2]:
## Read in the CCG under 75 CVD mortality file
cvd_mort = pd.read_excel('/content/CCG cvd mortality under 75.xls', sheet_name = 'Data')

SyntaxError: ignored

In [ ]:
## Copy your map making code from above to create a map for all CCGs in England showing CVD mortality in under 75s